In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import time
import sys

text = open('input.txt').read().lower() # doom lyrics
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

corpus length: 326073
total chars: 73
nb sequences: 108678
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
850/850 [==============================] - 264s 311ms/step - loss: 2.9073

----- diversity: 0.2
----- Generating with seed: "built in me
and building the empire high"
built in me
and building the empire high tan tan the te tee the te tame toe toon toe tout in tee tee the bon tout tere toe te te tout sole tor teut te te te tee tat tate tas the tout tou the tee te tee tat to te to the tout ans the te to the tee toun tout teut te toe to the tou tout tate the toe toe toad tor te tee tee tore tot te toate toe tee toe teete te in tou tee mhe te tout on sate the taut an te tee tee tout the toe toe toe tee t

----- diversity: 0.5
----- Generating with seed: "built in me
and building the empire high"
built in me
and building the empire highe toeme tolt wo teite the tor mate tine solice tot tot falte int tice tee te hatet sheretetler lele toe 

 in the ziegfried shirt with wide collar rigk the cleelp
wo amy in a shet fet to youll on the llear hip, the red the saald lurz, ann spem in tood a slally it lame
i mumed i'pe
bnype. gack onf he cast
sming peartt's calln boaproy in the weth u8llinn 'pe geans held onf)e
cheed gep of and uve they goat, snloidn wrat heon thes enmerin a get and ucw an, gramed that nlill thriil thrit "hargatt pas dock ne all wore the ho the begun i wick gut 

----- diversity: 1.2
----- Generating with seed: " in the ziegfried shirt with wide collar"
 in the ziegfried shirt with wide collars, lite nh, the bnue.s
feald?ped
cake jos
a adlipuing
oith me
he to0 wake depowh wont
lack
it jusin!, hig
wive terd dmitcey hlap i'm gilles uit ig gkipy vike wumd sait lavo, anoco byect micblit
eqtame stish
crot dalad hid wos'oq ip yaly bawd a'd, moeded rape
ithe fek wqh olm weoud
bpeem bows'ry hingt'l,! a bak, jep to notty spnyey a allla dyy mavepcuthb! mazolre ap tanigilwer, mytud, cro
herx) iln

------------------------

always threw me so the sare the sand the sare the sand the stranger the some the sare to the beand the could the slook to stark the sand the some so the seet so the seel the stare it the sear the sack the seed the seet and so the sand the sand the store the stall the sand the some so the some don't so the sout the stall so the sould the say so the store the sear the stall the sand the some to strang the say the 

----- diversity: 0.5
----- Generating with seed: "ther flossy, fatty juggy
always threw me"
ther flossy, fatty juggy
always threw ment she so the sullin' and conder the stomer come the don't the some dead to sapers is so she the sase and he inmer of lease
don't like a she send and the way and so so bry and and the slick shot what he chorser the here you a boing to hear the heall fund on the and the been what he hall back what to sell the beand a rall on the sair a hand the bick and a back he leee fof bay and a ses to they from

----- diversity: 1.0
----- Generating with seed: 

on real afv-iggle dy letticks and for me they got the brzozf book. me like yearding re. efpert with toongy the carn to the m

----- diversity: 1.2
----- Generating with seed: " the first tribe of men
and piled them i"
 the first tribe of men
and piled them it hakey a afmere
wo me s6ppa debbed up like
tath here star way, hear duen your the gues
fighter see for your we's.
hoald pod rhick
urpady lukies and but on the witcs
coing mitttle unwer
it's
shobly a remem, rillle
stenk
thills dirs grons
of 'evithics
onihe gicg ilce i viud hear ofu coned-if's shivel orloun, on a doam, a knoag. even nevan me be roge
(we so.e
sninings in the luck shurkou
sex o
gatkt

--------------------------------------------------
Iteration 12
850/850 [==============================] - 262s 308ms/step - loss: 1.9327

----- diversity: 0.2
----- Generating with seed: "t let's bang this madvillain music 'till"
t let's bang this madvillain music 'till the sick the say and a like the mind the say it the say to the bon't

do it till ya lips are numb, you you got the mast a supple day up of sull, she the chint is in the ampect was deat on the truck the be don't be could to bamm and a cripper in the live you do your need hole a use for the might to stor she pready and the get of in the slack of do so the smark and the sage come start of the mack got the street the same to rest be see it is the mother to rishe trues to started and a crace in the bea

----- diversity: 1.0
----- Generating with seed: "n' from
do it till ya lips are numb, you"
n' from
do it till ya lips are numb, you what he's ack shot barotrumatuld he scat gern fold to need on hig wathed flugg
paryer moze prosn
drow dig go wad'' warce-hattra, like to renie nimed of rap
cry to de or, go to crack a scomp crome baso coppo so aun uw pall wange just attecort
to the done theiss!
it tele come man chate sho i
kit, it's a spylice
and is go shint hard kik veds metting look
on bigsse to grers
and whre al bodin' ofs
the

----- diversity: 1.2
----- Gener

...

<ipython-input-1-79800aa58492>:55: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


e drups
hit namelove meat
from my mayes, you fuck, beatsta glack are crumps
all since my base
but blick, gone for dlinking!"
plave, off ball, and sue.codywold him or
smake till stakens, grandies trikbs, my hoard a chaunt out, son)
(
japer time timen come'm)
clower, you always humper on and sweets'ur opbo
proodin', h

--------------------------------------------------
Iteration 49
850/850 [==============================] - 232s 273ms/step - loss: 1.5175

----- diversity: 0.2
----- Generating with seed: "n it be i stayed away too long?
did you "
n it be i stayed away too long?
did you could the couldn the stress the fine for the stress shot me the stress the street the could her to the chese to the could the chick on the stress shot the trout in the stress so when he could the one in the choose than a fant to stay the stress the mic on the stress the stress the stress the stress the shot the stress the stress the stress the stress the shit
it's the poundeting to the chose to th

----- di

it's over, i'm go the mic of the diel she straight sell straights with the traight be the the old spot for who think the end it but it was who that so what he get the fact to the really check and respressions to the doom the start spark the street and doom's the ted and some matter with the goll the word the sinced that's no no bookies to steet it like stay for the polod
i'd but i'm but it's point of the never lik

----- diversity: 1.0
----- Generating with seed: "like a pigeon, i'm out
it's over, i'm go"
like a pigeon, i'm out
it's over, i'm go made he and seppistering you gotta got
our out, gonna fliet tape to you know my not time prain bount
hime of pt
schow and some" who futher whatest
mbnect
and all trime of allain by like the steelin', gaes, genough out but he's tobd-in some backs
only bout you what love through spot for the dead wede?
what i choor worth, these speader
the got some me, bite inbling to the too sleptet the never widd

----- diversity: 1.2
----- Generating with seed

skit awcochine to rocks of plackabes ane make orleda ilsmed with the firoce in pust, flood right when you mneps, hell new loom, dut no tore wrapport


--------------------------------------------------
Iteration 56
850/850 [==============================] - 368s 433ms/step - loss: 1.4664

----- diversity: 0.2
----- Generating with seed: " i missed it
word up, its crazy
why i go"
 i missed it
word up, its crazy
why i got the street that was the game is it out the street to the street and it's a cast to stand the beat stand of the mask and stay a started the street to the mic is the beat the street of the broke the street and the hold the street of the beat that don't be the beat the beat that make the least a stance for the street that is her and the beat that what a stand the chicks with the street the beat tha

----- diversity: 0.5
----- Generating with seed: " i missed it
word up, its crazy
why i go"
 i missed it
word up, its crazy
why i got black and check a lot of the weap that shi

who tapped micching zootiony"
yo" really gold imes of shit
how y'all stranger phes that ball
doom ope but the gan they ain't like than the beat called a slew us
she was wede rad with the clood to looks for you fuck it in ya
tide quate cracker
eaxyn
they pessing, beer in the back towation, a t'mes
cruelfry, toppy where hobaes, they with dangeror
cliquh”"
the why)
comppant
i best this slicker, ({stanted romelfi

----- diversity: 1.2
----- Generating with seed: "by the speaker and suddenly
who tapped m"
by the speaker and suddenly
who tapped mir fat well mide
be weme up at it last even every ack bootta humbly
the i cruel, that's the end
in a skept miswoes how as real cook 'se double couce bet i'm they not i'm seel'de

so - out twies everything it, hits emx


fuire, cut the laar
out
nine tester's as we head bees'p for deil, i'm a few bbackstoning blue
she frequene moinad yara laty sicf coeax
then got wear a girl the on make him, you cou
